### Using Our Filtered Exoplanet Dataset with Removed Outliers and Populated Columns (Samia's Code + Inara's Code + Ari's Manual Column Population)

In [92]:
# importing
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import warnings
warnings.filterwarnings('ignore')

In [93]:
# loading dataset
new_filtered_dataset = pd.read_excel('Exoplanet_filtered_dataset_removed_outliers_complete.xlsx', sheet_name='Data')

In [94]:
new_filtered_dataset

,planet_name,host_star_name,spectral_type,stellar_effective_temp_k,planet_equil_temp_k,orbital_period_day,orbit_semi_major_axis_au,planet_rad_earth_rad,planet_mass_earth_mass,stellar_rad_solar_rad,stellar_mass_solar_mass,distance_parsec,Eccentricity,stellar_surf_grav,declination_deg,discovery_method
0,K2-7 b,K2-7,G2V,5743.50,651,28.682907,0.183900,3.696667,NaN,1.563333,1.010000,738.553,NaN,3.820,-1.065855,Transit
1,HD 56414 b,HD 56414,A,8500.00,1133,29.049920,0.229000,3.710000,NaN,1.750000,1.890000,272.217,0.6800,NaN,-68.833340,Transit
2,KELT-17 b,KELT-17,A,7454.00,2087,3.080175,0.048810,17.094000,416.357300,1.650000,1.640000,226.500,NaN,4.220,13.735309,Transit
3,KIC 7917485 b,KIC 7917485,A,7067.00,NaN,840.000000,NaN,NaN,3750.394000,NaN,1.630000,1382.890,0.1500,4.000,43.632607,Pulsation Timing Variations
4,HD 102956 b,HD 102956,A,5011.00,NaN,6.494833,0.080567,NaN,300.873467,4.453333,1.643333,121.890,0.0450,3.410,57.640658,Radial Velocity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5625,KOI-3680 b,KOI-3680,NaN,5818.46,345,141.241634,0.530040,10.901400,613.411900,0.990000,0.998571,921.986,0.0992,4.450,45.309657,Transit
5626,KOI-4777.01,KOI-4777,NaN,3446.00,854.5,0.741600,0.010475,0.442500,99.200000,0.330000,0.330000,172.245,0.0000,4.912,42.032226,Transit
5627,MOA-2019-BLG-008L b,MOA-2019-BLG-008L,NaN,NaN,NaN,NaN,NaN,NaN,6912.767853,NaN,0.766667,2675.000,NaN,NaN,-29.989731,Microlensing
5628,OGLE-2006-BLG-284L A b,OGLE-2006-BLG-284L A,NaN,NaN,NaN,NaN,2.170000,NaN,144.000000,NaN,0.350000,4000.000,NaN,NaN,-29.136667,Microlensing


#### Understanding the Dataset now that we have populated the spectral_type column

In [95]:
# Finding the count of entries in the spectral_type column which contain missing values
total_count = len(new_filtered_dataset['spectral_type'])
non_missing_entries_count = new_filtered_dataset['spectral_type'].count()
missing_entries_count= total_count - non_missing_entries_count

print('The Total number of rows in the column (including missing values) :', total_count)
print('The number of entries in the column that have data is: ', non_missing_entries_count)
print('Number of missing entries: ', missing_entries_count)

The Total number of rows in the column (including missing values) : 5630
The number of entries in the column that have data is:  5532
Number of missing entries:  98


In [96]:
# Finding the entries in the spectral_tyoe column which contain / or -
# r is known as raw string and it treats the backslashes inside the string literally and not as escape characters. Without the r, you'd need to double the backslashes (e.g., \\)
# na=False handles the missing entries
contains_dash_or_slash_df= new_filtered_dataset[new_filtered_dataset['spectral_type'].str.contains(r'[/\-]', na=False)]

# List out the filtered entries
print("Entries that contain '/' or '-':")
print(contains_dash_or_slash_df)

# Get the count of such entries
count = len(contains_dash_or_slash_df)
print(f"Count of entries that contain '/' or '-': {count} entries")

Entries that contain '/' or '-':
               planet_name      host_star_name    spectral_type  \
5     KMT-2018-BLG-0087L b  KMT-2018-BLG-0087L          A0 - M2   
16             HIP 99770 b           HIP 99770            A5-A6   
31             HIP 39017 b           HIP 39017            A9/F0   
45                AB Aur b              AB Aur          B9 - A1   
49                KELT-9 b              KELT-9          B9.5-A0   
...                    ...                 ...              ...   
5333         Kepler-1245 b         Kepler-1245  M6V / M8V / M6V   
5334         Kepler-1245 c         Kepler-1245  M6V / M8V / M6V   
5509      CFHTWIR-Oph 98 b    CFHTWIR-Oph 98 A            M9-L1   
5525       NSVS 14256825 b       NSVS 14256825       sdOB / M V   
5526           V0391 Peg b           V0391 Peg        Sub-Dwarf   

      stellar_effective_temp_k planet_equil_temp_k  orbital_period_day  \
5                          NaN                 NaN                 NaN   
16            

#### Loading Dataset After Manually Removing rows where the stellar type contained binary stars, pulsars, eclipsing stars, brown dwrafs, white dwardf, and sub dwarfs. As well as setting one star type for entries with multiple star types

In [97]:
# loading dataset
new_filtered_dataset2 = pd.read_excel('Exoplanet_filtered_dataset_removed_outliers_complete.xlsx', sheet_name='New Data')

In [98]:
new_filtered_dataset2

,planet_name,host_star_name,spectral_type,stellar_effective_temp_k,planet_equil_temp_k,orbital_period_day,orbit_semi_major_axis_au,planet_rad_earth_rad,planet_mass_earth_mass,stellar_rad_solar_rad,stellar_mass_solar_mass,distance_parsec,Eccentricity,stellar_surf_grav,declination_deg,discovery_method
0,11 Com b,11 Com,G8 III,4808.000,NaN,324.620000,1.226000,NaN,5505.066163,16.380000,2.463333,93.18460,0.234500,2.380,17.793252,Radial Velocity
1,11 UMi b,11 UMi,K4 III,4276.500,NaN,516.219985,1.526667,NaN,3818.094733,26.935000,2.093333,125.32100,0.080000,1.765,71.823943,Radial Velocity
2,14 And b,14 And,K0 III,4850.500,NaN,186.300000,0.761667,NaN,1224.550433,11.275000,1.726667,75.43920,0.000000,2.590,39.235837,Radial Velocity
3,14 Her b,14 Her,K0 V,5296.985,NaN,1766.378417,2.814750,NaN,1642.383591,0.976667,0.927143,17.93230,0.362925,4.420,43.816362,Radial Velocity
4,16 Cyg B b,16 Cyg B,G2.5 V,5728.594,NaN,799.375000,1.662833,NaN,533.514528,1.140000,1.016000,21.13970,0.676033,4.344,50.516824,Radial Velocity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5589,XO-7 b,XO-7,G0 V,6250.000,1743,2.864139,0.044210,15.39,225.341470,1.480000,1.410000,234.14900,0.038000,4.250,85.233321,Transit
5590,YSES 2 b,YSES 2,K1 V,4749.000,NaN,NaN,115.000000,NaN,2336.038790,1.190000,1.100000,109.40600,NaN,NaN,-66.434574,Imaging
5591,YZ Cet b,YZ Cet,M4 V,3103.500,471,1.986177,0.015837,NaN,0.700000,0.166667,0.133333,3.71207,0.030000,5.170,-16.996243,Radial Velocity
5592,YZ Cet c,YZ Cet,M4 V,3103.500,410,2.290707,0.016887,NaN,0.900000,0.166667,0.133333,3.71207,0.030000,5.170,-16.996243,Radial Velocity


In [99]:
# Finding the count of entries in the spectral_type column which contain missing values
total_count2 = len(new_filtered_dataset2['spectral_type'])
non_missing_entries_count2 = new_filtered_dataset2['spectral_type'].count()
missing_entries_count2 = total_count2 - non_missing_entries_count2

print('The Total number of rows in the column (including missing values) :', total_count2)
print('The number of entries in the column that have data is: ', non_missing_entries_count2)
print('Number of missing entries: ', missing_entries_count2)

The Total number of rows in the column (including missing values) : 5594
The number of entries in the column that have data is:  5496
Number of missing entries:  98


#### For entries in specteral type which do not have a star type, set them at main sequence (V) as default

In [100]:
# Identify the entrie of the dataset which no not end with III, IV, and V
no_III_IV_V_data = new_filtered_dataset2[~new_filtered_dataset2['spectral_type'].str.endswith('V', na=False) & ~new_filtered_dataset2['spectral_type'].str.endswith(' V', na=False) & 
                             ~new_filtered_dataset2['spectral_type'].str.endswith('IV', na=False) & ~new_filtered_dataset2['spectral_type'].str.endswith(' IV', na=False) &
                             ~new_filtered_dataset2['spectral_type'].str.endswith('III', na=False) & ~new_filtered_dataset2['spectral_type'].str.endswith(' III', na=False) &
                             new_filtered_dataset2['spectral_type'].notna()]
no_III_IV_V_data.shape

(3589, 16)

In [101]:
no_III_IV_V_data.head(20)

,planet_name,host_star_name,spectral_type,stellar_effective_temp_k,planet_equil_temp_k,orbital_period_day,orbit_semi_major_axis_au,planet_rad_earth_rad,planet_mass_earth_mass,stellar_rad_solar_rad,stellar_mass_solar_mass,distance_parsec,Eccentricity,stellar_surf_grav,declination_deg,discovery_method
9,24 Sex b,24 Sex,K,5098.000,NaN,452.800000,1.333000,NaN,632.460000,4.900000,1.540000,72.0691,0.090000,3.500000,-0.902400,Radial Velocity
10,24 Sex c,24 Sex,K1,5098.000,NaN,883.000000,2.080000,NaN,273.320000,4.900000,1.540000,72.0691,0.290000,3.500000,-0.902400,Radial Velocity
12,2MASS J01033563-5515561 AB b,2MASS J01033563-5515561 A,M5.5/M6,NaN,NaN,NaN,84.000000,NaN,4131.790000,NaN,0.190000,NaN,NaN,NaN,-55.265829,Imaging
14,2MASS J02192210-3925225 b,2MASS J02192210-3925225,M6,3064.000,800,NaN,156.000000,16.141000,4417.837000,0.280000,0.110000,NaN,NaN,4.590000,-39.423077,Imaging
16,2MASS J03590986+2009361 b,2MASS J03590986+2009361,M,NaN,2369,NaN,572.000000,NaN,5720.911330,NaN,NaN,117.4350,NaN,NaN,20.159976,Imaging
17,2MASS J04414489+2301513 b,2MASS J04414489+2301513,M8.5,2936.000,900,NaN,15.000000,NaN,2383.600000,NaN,0.020000,NaN,NaN,NaN,23.030854,Imaging
18,2MASS J11550485-7919108 b,2MASS J11550485-7919108,M,NaN,2506,NaN,587.000000,NaN,6356.568140,NaN,NaN,100.8620,NaN,NaN,-79.319756,Imaging
19,2MASS J12073346-3932539 b,2MASS J12073346-3932539,M8,2825.000,1200,NaN,50.500000,NaN,1430.075000,0.220000,0.105000,64.3080,NaN,5.030000,-39.548440,Imaging
20,2MASS J19383260+4603591 b,2MASS J19383260+4603591,B,29564.000,305,411.000000,0.920000,NaN,597.518920,0.200000,0.480000,396.3320,0.330000,5.510000,46.066408,Eclipse Timing Variations
21,2MASS J21252752-8138278 b,2MASS J21252752-8138278,M,NaN,1616,NaN,7493.000000,NaN,3813.940880,NaN,NaN,34.1154,NaN,NaN,-81.641489,Imaging


In [102]:
# Append 'V' to the end of these entries
new_filtered_dataset2.loc[no_III_IV_V_data.index, 'spectral_type'] = no_III_IV_V_data['spectral_type'] + ' V'

# Verify the changes 
new_filtered_dataset2.head(20)

,planet_name,host_star_name,spectral_type,stellar_effective_temp_k,planet_equil_temp_k,orbital_period_day,orbit_semi_major_axis_au,planet_rad_earth_rad,planet_mass_earth_mass,stellar_rad_solar_rad,stellar_mass_solar_mass,distance_parsec,Eccentricity,stellar_surf_grav,declination_deg,discovery_method
0,11 Com b,11 Com,G8 III,4808.000,NaN,324.620000,1.226000,NaN,5505.066163,16.380000,2.463333,93.1846,0.234500,2.380,17.793252,Radial Velocity
1,11 UMi b,11 UMi,K4 III,4276.500,NaN,516.219985,1.526667,NaN,3818.094733,26.935000,2.093333,125.3210,0.080000,1.765,71.823943,Radial Velocity
2,14 And b,14 And,K0 III,4850.500,NaN,186.300000,0.761667,NaN,1224.550433,11.275000,1.726667,75.4392,0.000000,2.590,39.235837,Radial Velocity
3,14 Her b,14 Her,K0 V,5296.985,NaN,1766.378417,2.814750,NaN,1642.383591,0.976667,0.927143,17.9323,0.362925,4.420,43.816362,Radial Velocity
4,16 Cyg B b,16 Cyg B,G2.5 V,5728.594,NaN,799.375000,1.662833,NaN,533.514528,1.140000,1.016000,21.1397,0.676033,4.344,50.516824,Radial Velocity
5,17 Sco b,17 Sco,K3 III,4157.000,1300,578.380000,1.450000,NaN,1373.018720,25.920000,1.220000,124.9530,0.060000,1.700,-11.837791,Radial Velocity
6,18 Del b,18 Del,G6 III,4979.500,NaN,988.075000,2.538667,NaN,3109.015380,8.650000,2.166667,76.2220,0.052000,2.845,10.839138,Radial Velocity
7,1RXS J160929.1-210524 b,1RXS J160929.1-210524,K7 V,4043.250,1750,NaN,330.000000,18.647,3135.750000,1.330000,0.792500,139.1350,NaN,4.000,-21.083140,Imaging
8,24 Boo b,24 Boo,G3 IV,4854.500,NaN,30.340300,0.192000,NaN,284.933890,11.415000,1.020000,95.9863,0.037000,2.350,49.844649,Radial Velocity
9,24 Sex b,24 Sex,K V,5098.000,NaN,452.800000,1.333000,NaN,632.460000,4.900000,1.540000,72.0691,0.090000,3.500,-0.902400,Radial Velocity


In [103]:
# Saving dataset
#new_filtered_dataset2.to_csv('Final_Exoplanet_FIltered_Dataset_Removed_Outliers.csv', index=False)

#### Host Star Classification

In [104]:
# loading dataset
exoplanet_dataset = pd.read_excel('Exoplanet_filtered_dataset_removed_outliers_complete.xlsx', sheet_name='Final New Data')

In [105]:
exoplanet_dataset.columns

Index(['planet_name', 'host_star_name', 'spectral_type',
       'stellar_effective_temp_k', 'planet_equil_temp_k', 'orbital_period_day',
       'orbit_semi_major_axis_au', 'planet_rad_earth_rad',
       'planet_mass_earth_mass', 'stellar_rad_solar_rad',
       'stellar_mass_solar_mass', 'distance_parsec', 'Eccentricity',
       'stellar_surf_grav', 'declination_deg', 'discovery_method'],
      dtype='object')

In [106]:
exoplanet_dataset.shape

(5594, 16)

In [107]:
# Finding the count of entries in the spectral_type column which contain missing values
total_count3 = len(exoplanet_dataset['spectral_type'])
non_missing_entries_count3 = exoplanet_dataset['spectral_type'].count()
missing_entries_count3 = total_count3 - non_missing_entries_count3

print('The Total number of rows in the column (including missing values) :', total_count3)
print('The number of entries in the column that have data is: ', non_missing_entries_count3)
print('Number of missing entries: ', missing_entries_count3)

The Total number of rows in the column (including missing values) : 5594
The number of entries in the column that have data is:  5496
Number of missing entries:  98


In [108]:
# filtering dataset to only contain main sequence star type
# na=False handles the missing values
# dataset['spectral_type'].str.endswith('V', na=False) means: filter the dataset to include any star type that ends with 'V' and creates a boolean Series where True indicates that the Star Type ends with 'V' and False otherwise.
# ~dataset['spectral_type'].str.endswith('IV', na=False) means: filter out any star type that ends with 'IV' and inverts the boolean Series so that True indicates that the Star Type does not end with 'IV' and False indicates that it does.
main_sequence_exoplanet = exoplanet_dataset[exoplanet_dataset['spectral_type'].str.endswith('V', na=False) &
                                            exoplanet_dataset['spectral_type'].str.endswith(' V', na=False) & 
                                            ~exoplanet_dataset['spectral_type'].str.endswith('IV', na=False) & 
                                            ~exoplanet_dataset['spectral_type'].str.endswith(' IV', na=False) &
                                            new_filtered_dataset2['spectral_type'].notna()]
main_sequence_exoplanet.shape

(4681, 16)

In [109]:
main_sequence_exoplanet.sample(15)

,planet_name,host_star_name,spectral_type,stellar_effective_temp_k,planet_equil_temp_k,orbital_period_day,orbit_semi_major_axis_au,planet_rad_earth_rad,planet_mass_earth_mass,stellar_rad_solar_rad,stellar_mass_solar_mass,distance_parsec,Eccentricity,stellar_surf_grav,declination_deg,discovery_method
1484,K2-195 b,K2-195,G4 V,5725.310000,720,15.853477,0.120800,3.090000,NaN,0.958000,0.976667,315.0140,NaN,4.476667,-8.509536,Transit
2866,Kepler-174 c,Kepler-174,K3 V,4717.666667,352.4,44.000426,0.206950,1.534000,NaN,0.678333,0.673000,384.7570,0.000000,4.610909,43.832005,Transit
2668,Kepler-159 c,Kepler-159,M0 V,4257.363636,285,43.592538,0.200173,2.812778,NaN,0.592727,0.598889,373.7440,0.000000,4.687778,40.868734,Transit
1936,Kepler-1053 b,Kepler-1053,K4 V,4358.366000,835.4,2.414351,0.029460,0.853429,NaN,0.603000,0.633333,151.1290,0.000000,4.678889,39.127366,Transit
2658,Kepler-1581 b,Kepler-1581,G0 V,5974.925000,1114.75,6.283839,0.068000,0.767333,NaN,1.301250,1.081429,493.1750,0.000000,4.251429,39.603623,Transit
4526,Kepler-937 c,Kepler-937,G0 V,6005.967500,377,153.345937,0.554275,2.431000,NaN,1.215000,1.001429,1571.4600,0.000000,4.288571,46.714616,Transit
4938,OGLE-TR-111 b,OGLE-TR-111,K2 V,4985.250000,1026.5,4.014782,0.046907,11.403000,179.573013,0.855000,0.820000,1068.3300,0.270000,4.510000,-61.405698,Transit
2589,Kepler-1530 c,Kepler-1530,G7 V,5508.412222,1167,5.323283,0.060000,2.200833,NaN,1.038889,0.965000,489.8760,0.000000,4.387500,48.165153,Transit
5096,TOI-1728 b,TOI-1728,M0 V,3980.000000,767,3.491477,0.039100,5.050000,26.780000,0.620000,0.650000,60.7980,0.057000,4.660000,64.797153,Transit
3917,Kepler-487 d,Kepler-487,G7 V,5570.500000,1262.6,2.418403,0.034000,2.615833,NaN,0.888750,0.908750,872.3430,0.000000,4.498750,41.222003,Transit


In [110]:
# filtering dataset to only contain subgiant star type
subgiant_exoplanet = exoplanet_dataset[exoplanet_dataset['spectral_type'].str.endswith('IV', na=False) & 
                                       exoplanet_dataset['spectral_type'].str.endswith(' IV', na=False) &
                                       new_filtered_dataset2['spectral_type'].notna()]
subgiant_exoplanet.shape

(95, 16)

In [111]:
subgiant_exoplanet.sample(15)

,planet_name,host_star_name,spectral_type,stellar_effective_temp_k,planet_equil_temp_k,orbital_period_day,orbit_semi_major_axis_au,planet_rad_earth_rad,planet_mass_earth_mass,stellar_rad_solar_rad,stellar_mass_solar_mass,distance_parsec,Eccentricity,stellar_surf_grav,declination_deg,discovery_method
898,HD 212771 b,HD 212771,G8 IV,5064.333333,NaN,375.766663,1.190000,NaN,887.794800,4.940000,1.706667,111.1550,0.099000,3.440000,-17.264105,Radial Velocity
661,HD 142 b,HD 142,G1 IV,6173.000000,NaN,347.608485,1.026000,NaN,384.481920,1.470000,1.207500,26.1853,0.239500,4.400000,-49.075362,Radial Velocity
597,HD 124330 b,HD 124330,G4 IV,5873.000000,NaN,270.660000,0.860000,NaN,238.371310,NaN,1.150000,60.1977,0.340000,4.240000,54.408973,Radial Velocity
737,HD 1605 c,HD 1605,K1 IV,4836.000000,NaN,2130.000000,3.552000,NaN,1130.272300,3.645000,1.320000,88.8047,0.098500,3.400000,30.975061,Radial Velocity
627,HD 134606 c,HD 134606,G6 IV,5576.000000,NaN,58.883000,0.300700,NaN,11.310000,1.160000,1.050000,26.7909,0.055000,4.330000,-70.520333,Radial Velocity
826,HD 190360 b,HD 190360,G6 IV,5554.780000,150,3015.767109,4.092500,NaN,483.451726,1.156000,0.980000,16.0069,0.346800,4.336000,29.894541,Radial Velocity
624,HD 134060 b,HD 134060,G3 IV,5966.000000,NaN,3.269600,0.044400,NaN,10.100000,NaN,1.090000,24.0125,0.450000,4.430000,-61.422375,Radial Velocity
1110,HD 5608 b,HD 5608,K0 IV,4866.750000,NaN,783.449995,1.897750,NaN,468.719328,5.290000,1.587500,58.1105,0.136500,3.190000,33.950577,Radial Velocity
516,HD 106270 b,HD 106270,G5 IV,5571.333333,NaN,2135.000000,3.662333,NaN,3361.809785,2.580000,1.380000,93.8080,0.249250,3.793333,-9.513564,Radial Velocity
790,HD 179079 b,HD 179079,G5 IV,5686.666667,NaN,14.477000,0.121000,NaN,28.342410,1.643333,1.273333,69.7116,0.094667,4.110000,-2.638770,Radial Velocity


In [112]:
# filtering dataset to only contain redgiant star type
redgiant_exoplanet = exoplanet_dataset[exoplanet_dataset['spectral_type'].str.endswith('III', na=False) & 
                                       exoplanet_dataset['spectral_type'].str.endswith(' III', na=False) &
                                       new_filtered_dataset2['spectral_type'].notna()]
redgiant_exoplanet.shape

(111, 16)

In [113]:
redgiant_exoplanet.sample(15)

,planet_name,host_star_name,spectral_type,stellar_effective_temp_k,planet_equil_temp_k,orbital_period_day,orbit_semi_major_axis_au,planet_rad_earth_rad,planet_mass_earth_mass,stellar_rad_solar_rad,stellar_mass_solar_mass,distance_parsec,Eccentricity,stellar_surf_grav,declination_deg,discovery_method
63,BD+20 274 b,BD+20 274,K5 III,4296.00,NaN,578.200000,1.300000,NaN,1334.886000,17.300000,0.800000,1322.51000,0.210000,1.9900,21.005269,Radial Velocity
706,HD 154391 b,HD 154391,K1 III,4909.00,NaN,5163.000000,7.460000,NaN,2892.238500,8.560000,2.070000,100.94700,0.200000,2.8900,60.648923,Radial Velocity
887,HD 208527 b,HD 208527,M1 III,4035.00,NaN,875.500000,2.100000,NaN,3146.400000,51.100000,1.600000,312.20200,0.080000,1.4000,21.239922,Radial Velocity
1116,HD 60292 b,HD 60292,K0 III,4348.00,NaN,495.400000,1.500000,NaN,2065.895000,27.000000,1.700000,315.21100,0.270000,1.9000,66.235052,Radial Velocity
87,bet UMi b,bet UMi,K4 III,4126.00,NaN,522.300000,1.400000,NaN,1938.700000,38.300000,1.400000,38.77472,0.190000,1.5000,74.155548,Radial Velocity
1833,kap CrB b,kap CrB,K0 III,4880.00,NaN,1257.132727,2.700909,NaN,592.694685,4.837000,1.607143,30.06510,0.101909,3.3500,35.655882,Radial Velocity
1006,HD 29399 b,HD 29399,K1 III,4845.00,NaN,892.700000,1.913000,NaN,498.990600,4.500000,1.170000,44.15180,0.050000,3.2500,-62.823758,Radial Velocity
979,HD 25723 b,HD 25723,K1 III,4766.00,NaN,457.010000,1.490000,NaN,794.571020,13.760000,2.120000,114.81600,0.040000,2.5000,-12.792224,Radial Velocity
1033,HD 33844 c,HD 33844,K0 III,4861.00,NaN,916.000000,2.240000,NaN,556.202500,5.290000,1.780000,105.77900,0.130000,3.2400,-14.950978,Radial Velocity
811,HD 18438 b,HD 18438,M2.5 III,3860.00,NaN,803.000000,2.100000,NaN,6674.396550,88.470000,1.840000,224.14600,0.100000,0.9000,79.418580,Radial Velocity
